<a href="https://colab.research.google.com/github/meizhong986/WhisperJAV/blob/main/notebook/WhisperJAV_colab_parallel_expert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<a href="https://kaggle.com/kernels/welcome?src=https://github.com/meizhong986/WhisperJAV/blob/main/notebook/WhisperJAV_colab_parallel_expert.ipynb" target="_parent"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Kaggle"/></a>

# WhisperJAV Ensemble Dual-GPU Edition (v1.7.5)

**Universal** - Auto-detects Kaggle T4x2 for parallel processing, falls back to single-GPU on Colab

| Platform | Logic | Storage |
|----------|-------|---------|
| **Kaggle** | **Parallel** (2x T4) | **Input**: `/kaggle/input` (Dataset) <br> **Output**: `/kaggle/working` (Artifacts) |
| **Colab** | **Sequential** (1x GPU) | **Input/Output**: Google Drive |

| Option | What it controls |
|--------|------------------|
| **Scene Detection** | How to split audio into chunks (auditok, silero, semantic) |
| **Speech Segmenter** | How to detect speech in audio (silero, ten) |
| **Speech Enhancer** | Audio cleanup for noisy sources (ffmpeg-dsp, clearvoice, etc.) |
| **Model** | Which AI model to use (large-v2, large-v3, turbo, kotoba) |

---
<div style="font-size: 11px; line-height: 1.4;">
1. <b>Setup Files:</b><br>
&nbsp;&nbsp;&nbsp;• <b>Colab:</b> Upload videos to <code>Google Drive/WhisperJAV/</code><br>
&nbsp;&nbsp;&nbsp;• <b>Kaggle:</b> Add your videos as a Dataset (recommended) or upload to Input.<br>
2. Run <b>Step 1: Expert Configuration</b> (required)<br>
3. Run <b>Step 2: Two-Pass Transcribe</b> (auto-detects platform)<br>
4. Run <b>Step 3: AI Translation</b> (supports Kaggle Secrets/Env/Keys)
</div>

<small>The notebook will automatically disconnect (Colab) or finish session (Kaggle) when done.</small>


In [ ]:
#@title Step 1: Expert Configuration { display-mode: "form" }

#@markdown ## 📁 Files & Output
folder_name = "WhisperJAV" #@param {type:"string"}
subtitle_language = "Japanese" #@param ["Japanese", "English (auto-translate)", "English (AI translate)"]
spoken_language = "Japanese" #@param ["Japanese", "Chinese", "English", "Korean"]

#@markdown ---
#@markdown ## 1️⃣ Pass 1 Configuration (GPU 0)
pass1_quality = "balanced" #@param ["faster", "fast", "balanced", "fidelity", "transformers"]
pass1_sensitivity = "aggressive" #@param ["conservative", "balanced", "aggressive"]
pass1_model = "large-v2" #@param ["automatic", "large-v2", "large-v3", "turbo", "kotoba-bilingual", "kotoba-v2.0", "kotoba-v2.1", "kotoba-v2.2"]

#@markdown **Expert Audio Setup (Pass 1)**
pass1_scene_detector = "semantic" #@param ["automatic", "auditok", "silero", "semantic"]
pass1_speech_segmenter = "ten" #@param ["automatic", "silero", "ten", "none"]
pass1_speech_enhancer = "none" #@param ["none", "ffmpeg-dsp", "clearvoice", "zipenhancer", "bs-roformer"]
#@markdown <font size="1">auditok=energy (fast), silero=VAD, semantic=texture (complex audio) | enhancer: ffmpeg-dsp(no GPU), clearvoice(48k), bs-roformer(vocal)</font>

#@markdown **FFmpeg Filters (Pass 1)** *(only if enhancer is ffmpeg-dsp)*
pass1_ffmpeg_amplify = True #@param {type:"boolean"}
pass1_ffmpeg_loudnorm = False #@param {type:"boolean"}
pass1_ffmpeg_compress = False #@param {type:"boolean"}
pass1_ffmpeg_highpass = False #@param {type:"boolean"}

#@markdown ---
#@markdown ## 2️⃣ Pass 2 Configuration (GPU 1)
pass2_quality = "fidelity" #@param ["faster", "fast", "balanced", "fidelity", "transformers"]
pass2_sensitivity = "balanced" #@param ["conservative", "balanced", "aggressive"]
pass2_model = "turbo" #@param ["automatic", "large-v2", "large-v3", "turbo", "kotoba-bilingual", "kotoba-v2.0", "kotoba-v2.1", "kotoba-v2.2"]

#@markdown **Expert Audio Setup (Pass 2)**
pass2_scene_detector = "semantic" #@param ["automatic", "auditok", "silero", "semantic"]
pass2_speech_segmenter = "ten" #@param ["automatic", "silero", "ten", "none"]
pass2_speech_enhancer = "ffmpeg-dsp" #@param ["none", "ffmpeg-dsp", "clearvoice", "zipenhancer", "bs-roformer"]

#@markdown **FFmpeg Filters (Pass 2)** *(only if enhancer is ffmpeg-dsp)*
pass2_ffmpeg_amplify = True #@param {type:"boolean"}
pass2_ffmpeg_loudnorm = False #@param {type:"boolean"}
pass2_ffmpeg_compress = False #@param {type:"boolean"}
pass2_ffmpeg_highpass = False #@param {type:"boolean"}

#@markdown ---
#@markdown ## 🔗 Merge Strategy
merge_method = "prefer first pass" #@param ["automatic", "keep all", "prefer first pass", "prefer second pass"]

#@markdown ---
#@markdown ## 🤖 AI Translation *(if selected)*
translation_service = "deepseek" #@param ["deepseek", "openrouter", "gemini", "claude", "gpt"]
api_key = "" #@param {type:"string"}
translation_style = "standard" #@param ["standard", "explicit"]

#@markdown ---
#@markdown ## ⚙️ Session
opening_credit = "" #@param {type:"string"}
closing_credit = "Subs by WhisperJAV" #@param {type:"string"}
auto_disconnect = True #@param {type:"boolean"}

# ═══════════════════════════════════════════
# CONFIGURATION LOGIC
# ═══════════════════════════════════════════

# Mapping dictionaries
combine_map = {"automatic": "smart_merge", "keep all": "full_merge",
               "prefer first pass": "pass1_primary", "prefer second pass": "pass2_primary"}
language_map = {"Japanese": "native", "English (auto-translate)": "direct-to-english",
                "English (AI translate)": "llm"}
tone_map = {"standard": "standard", "explicit": "pornify"}
spoken_language_map = {"Japanese": "japanese", "Chinese": "chinese", "English": "english", "Korean": "korean"}

# Model mapping (None = use pipeline default)
model_map = {
    "automatic": None,
    "large-v2": "large-v2",
    "large-v3": "large-v3",
    "turbo": "turbo",
    "kotoba-bilingual": "kotoba-tech/kotoba-whisper-bilingual-v1.0",
    "kotoba-v2.0": "kotoba-tech/kotoba-whisper-v2.0",
    "kotoba-v2.1": "kotoba-tech/kotoba-whisper-v2.1",
    "kotoba-v2.2": "kotoba-tech/kotoba-whisper-v2.2"
}

# Define model compatibility:
KOTOBA_MODELS = {"kotoba-bilingual", "kotoba-v2.0", "kotoba-v2.1", "kotoba-v2.2"}
LEGACY_PIPELINES = {"faster", "fast", "balanced", "fidelity"}

# Auto-correct incompatible model-pipeline combinations
warnings_list = []

# Check Pass 1 compatibility
if pass1_model in KOTOBA_MODELS and pass1_quality in LEGACY_PIPELINES:
    warnings_list.append(f"Pass 1: {pass1_model} requires 'transformers' pipeline. Auto-correcting from '{pass1_quality}' to 'transformers'.")
    pass1_quality = "transformers"

# Check Pass 2 compatibility
if pass2_model in KOTOBA_MODELS and pass2_quality in LEGACY_PIPELINES:
    warnings_list.append(f"Pass 2: {pass2_model} requires 'transformers' pipeline. Auto-correcting from '{pass2_quality}' to 'transformers'.")
    pass2_quality = "transformers"

# Memory warning
heavy_enhancers = {'clearvoice', 'bs-roformer', 'zipenhancer'}
if pass1_speech_enhancer in heavy_enhancers and pass2_speech_enhancer in heavy_enhancers:
    warnings_list.append("Using GPU-based enhancement on both passes may cause OOM on T4 GPU (Sequential Mode). Suggest using ffmpeg-dsp for one pass.")

# Helpers
def build_ffmpeg_filters(amplify, loudnorm, compress, highpass):
    """Combine selected FFmpeg filters into comma-separated string."""
    filters = []
    if amplify: filters.append("amplify")
    if loudnorm: filters.append("loudnorm")
    if compress: filters.append("compress")
    if highpass: filters.append("highpass")
    return ",".join(filters) if filters else None

def map_value(val):
    return None if val == "automatic" else val

def map_segmenter(val):
    return "none" if val == "none" else map_value(val)

# Unified Config Construction
WHISPERJAV_CONFIG = {
    'pass1_pipeline': pass1_quality,
    'pass1_sensitivity': pass1_sensitivity,
    'pass1_speech_segmenter': map_segmenter(pass1_speech_segmenter),
    'pass1_model': model_map[pass1_model],
    'pass2_pipeline': pass2_quality,
    'pass2_sensitivity': pass2_sensitivity,
    'pass2_speech_segmenter': map_segmenter(pass2_speech_segmenter),
    'pass2_model': model_map[pass2_model],
    'merge_strategy': combine_map[merge_method],
    'folder_name': folder_name,
    'subtitle_language': language_map[subtitle_language],
    'language': spoken_language_map[spoken_language],
    'translation_service': translation_service,
    'api_key': api_key,
    'translation_style': tone_map[translation_style],
    'opening_credit': opening_credit,
    'closing_credit': closing_credit,
    'auto_disconnect': auto_disconnect,
    # Compatibility checks for Step 2
    '_pass1_quality': pass1_quality,
    '_pass1_sensitivity': pass1_sensitivity,
    '_pass1_speech_segmenter': pass1_speech_segmenter,
    '_pass1_model': pass1_model,
    '_pass2_quality': pass2_quality,
    '_pass2_sensitivity': pass2_sensitivity,
    '_pass2_speech_segmenter': pass2_speech_segmenter,
    '_pass2_model': pass2_model,
    '_merge_method': merge_method,
    '_subtitle_language': subtitle_language,
    '_translation_style': translation_style,
}

WHISPERJAV_EXPERT_CONFIG = {
    # Pass 1 Expert
    'pass1_scene_detector': map_value(pass1_scene_detector),
    'pass1_speech_segmenter': map_segmenter(pass1_speech_segmenter), # Now unified
    'pass1_speech_enhancer': None if pass1_speech_enhancer == "none" else pass1_speech_enhancer,
    'pass1_ffmpeg_filters': build_ffmpeg_filters(pass1_ffmpeg_amplify, pass1_ffmpeg_loudnorm, pass1_ffmpeg_compress, pass1_ffmpeg_highpass) if pass1_speech_enhancer == "ffmpeg-dsp" else None,
    # Pass 2 Expert
    'pass2_scene_detector': map_value(pass2_scene_detector),
    'pass2_speech_segmenter': map_segmenter(pass2_speech_segmenter), # Now unified
    'pass2_speech_enhancer': None if pass2_speech_enhancer == "none" else pass2_speech_enhancer,
    'pass2_ffmpeg_filters': build_ffmpeg_filters(pass2_ffmpeg_amplify, pass2_ffmpeg_loudnorm, pass2_ffmpeg_compress, pass2_ffmpeg_highpass) if pass2_speech_enhancer == "ffmpeg-dsp" else None,
    # Display helpers
    '_pass1_scene_detector': pass1_scene_detector,
    '_pass1_speech_enhancer': pass1_speech_enhancer,
    '_pass2_scene_detector': pass2_scene_detector,
    '_pass2_speech_enhancer': pass2_speech_enhancer,
}

from IPython.display import display, HTML

# Display warnings
for warning in warnings_list:
    display(HTML(f'<div style="padding:6px 10px;background:#fef9c3;border-radius:4px;font-size:10px;margin-bottom:4px"><b>⚠️ Auto-corrected:</b> {warning}</div>'))

# Build status display
p1_info = f"{pass1_quality}"
if pass1_speech_segmenter != "automatic":
    p1_info += f"/{pass1_speech_segmenter}"
if pass1_model != "automatic":
    p1_info += f"/{pass1_model}"

p2_info = f"{pass2_quality}"
if pass2_speech_segmenter != "automatic":
    p2_info += f"/{pass2_speech_segmenter}"
if pass2_model != "automatic":
    p2_info += f"/{pass2_model}"

display(HTML(f'<div style="padding:10px;background:#e0f2fe;border-radius:4px;font-size:11px">'
             f'<b>Parallel Configuration Loaded</b><br>'
             f'Pass 1: {p1_info} | Pass 2: {p2_info}<br>'
             f'Merge: {merge_method} | Folder: {folder_name} | Language: {spoken_language}'
             f'</div>'))

In [ ]:
#@title Step 1.5: TEN prerequisites + sanity check { display-mode: "form" }
#@markdown Run this **before Step 2** if you use `ten` as the speech segmenter.

import os, shutil, subprocess, sys, textwrap
from pathlib import Path

def _apt_prefix() -> str:
    """Use sudo only when needed + available."""
    try:
        if hasattr(os, "geteuid") and os.geteuid() == 0:
            return ""
    except Exception:
        pass
    return "sudo " if shutil.which("sudo") else ""

def run(cmd: str) -> None:
    print(f"$ {cmd}")
    r = subprocess.run(cmd, shell=True, text=True, capture_output=True)
    if r.returncode != 0:
        print("--- stdout (tail) ---")
        print((r.stdout or "")[-4000:])
        print("--- stderr (tail) ---")
        print((r.stderr or "")[-4000:])
        raise RuntimeError(f"Command failed (rc={r.returncode}): {cmd}")

print(f"Python: {sys.version.split()[0]} | exe: {sys.executable}")

# 1) Install runtime libs required by TEN VAD (system packages)
prefix = _apt_prefix()
try:
    run(prefix + "apt-get update -qq")
    run(prefix + "apt-get install -y -qq libc++1 libc++abi1")
except Exception:
    print("\nApt install failed. On Kaggle you may need to enable Internet, or your image may block apt.")
    raise

# 2) Install the Python package into THIS kernel's Python environment
#    (This fixes the common issue where `pip` points to a different Python than the notebook kernel.)
run(f"{sys.executable} -m pip install -q --upgrade pip")
run(f"{sys.executable} -m pip install -q ten-vad")

# 3) Prove the loader can see libc++ from this runtime
run("ldconfig -p | egrep 'libc\\+\\+\\.so\\.1|libc\\+\\+abi\\.so\\.1' || true")

# 4) Prove Python can load the shared libs, then import/init TEN
code = textwrap.dedent(r"""
    import ctypes
    for lib in ('libc++.so.1','libc++abi.so.1'):
        try:
            ctypes.CDLL(lib)
            print('OK ctypes load', lib)
        except OSError as e:
            print('FAIL ctypes load', lib, ':', e)
    
    try:
        import ten_vad
        print('ten_vad import OK:', ten_vad.__file__)
        from ten_vad import TenVad
        _ = TenVad()
        print('TenVad() OK')
    except Exception as e:
        print('TEN IMPORT/INIT FAILED:', repr(e))
""")
r = subprocess.run([sys.executable, "-c", code], text=True, capture_output=True)
print(r.stdout)
if r.returncode != 0 and r.stderr:
    print(r.stderr)

print("\nIf you still see TEN failures in Step 2 after this cell passes, copy/paste the exact 'TEN IMPORT/INIT FAILED' line here — it will be a different root cause than missing libc++ or missing the Python package.")

In [ ]:
#@title Step 1.6: Preflight imports (FAIL FAST) { display-mode: "form" }
#@markdown This cell fails immediately if critical imports/initializations are broken. Run it before Step 2.

import importlib, shutil, subprocess, sys, time
from pathlib import Path

FAIL_FAST_PREFLIGHT = True

def must(condition: bool, msg: str):
    if not condition:
        raise SystemExit(f"Preflight failed: {msg}")

def which(cmd: str) -> str:
    p = shutil.which(cmd)
    return p or ""

print(f"Kernel Python: {sys.executable}")
print(f"Python version: {sys.version.split()[0]}")

# 1) Core CLI package must be importable in THIS kernel
try:
    import whisperjav
    print("OK import whisperjav:", whisperjav.__file__)
except Exception as e:
    raise SystemExit(f"Preflight failed: cannot import whisperjav in kernel env: {e!r}")

# 2) TEN segmenter optional, but if you plan to use it, it must import + init
try:
    import ten_vad
    from ten_vad import TenVad
    _ = TenVad()
    print("OK TEN: ten_vad import + TenVad()")
except Exception as e:
    # If you don't use TEN, you can ignore; if you do, treat as fatal.
    if FAIL_FAST_PREFLIGHT:
        raise SystemExit(f"Preflight failed: TEN not usable (import/init): {e!r}")
    print("WARN: TEN not usable:", repr(e))

# 3) GPU stack sanity (best effort)
try:
    import torch
    print("OK torch:", torch.__version__)
    print("CUDA available:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("GPU:", torch.cuda.get_device_name(0))
except Exception as e:
    # Not always fatal (CPU mode exists), but for Kaggle/Colab GPU notebooks, treat as fatal.
    if FAIL_FAST_PREFLIGHT:
        raise SystemExit(f"Preflight failed: torch/CUDA stack issue: {e!r}")
    print("WARN: torch/CUDA issue:", repr(e))

# 4) System tools needed by pipeline
ffmpeg = which("ffmpeg")
must(bool(ffmpeg), "ffmpeg not found on PATH")
print("OK ffmpeg:", ffmpeg)

# 5) Confirm that 'whisperjav' on PATH matches the kernel env (diagnostic)
whisperjav_bin = which("whisperjav")
print("whisperjav on PATH:", whisperjav_bin or "<not found>")
if whisperjav_bin:
    try:
        out = subprocess.run([whisperjav_bin, "--help"], text=True, capture_output=True, timeout=10)
        print("whisperjav --help rc=", out.returncode)
    except Exception as e:
        print("WARN: unable to run whisperjav --help:", repr(e))

# Contract for Step 2: stamp a kernel-specific preflight token
WHISPERJAV_PREFLIGHT = {
    "ok": True,
    "python": sys.executable,
    "python_version": sys.version.split()[0],
    "timestamp": int(time.time()),
}
print("\nPreflight OK. Token saved in WHISPERJAV_PREFLIGHT.")

In [ ]:
#@title Step 2: Two-Pass Transcribe { display-mode: "form" }
#@markdown Connect Drive (Colab) or Setup Paths (Kaggle) → Install → Run passes → Merge results

import os, sys, subprocess, shlex, time, re, shutil
from pathlib import Path
from IPython.display import display, clear_output
from IPython.core.display import HTML
from dataclasses import dataclass
from typing import List, Dict, Any, Tuple, Optional

def status(msg, ok=True):
    icon = "✓" if ok else "✗"
    print(f"{icon} {msg}")

def section(title):
    print(f"\n{'─'*50}\n{title}\n{'─'*50}")

# ═══════════════════════════════════════════
# FAIL-FAST GUARD: Require Step 1.6 token
# ═══════════════════════════════════════════
if 'WHISPERJAV_PREFLIGHT' not in globals() or not isinstance(WHISPERJAV_PREFLIGHT, dict) or not WHISPERJAV_PREFLIGHT.get('ok'):
    display(HTML('<div style="padding:10px;background:#fee2e2;border-radius:4px;border-left:4px solid #ef4444;color:#991b1b;"><b>🛑 Preflight not run</b><br>Run <b>Step 1.6: Preflight imports (FAIL FAST)</b> first. This notebook refuses to start Step 2 without it.</div>'))
    raise SystemExit()
if WHISPERJAV_PREFLIGHT.get('python') != sys.executable:
    display(HTML(f'<div style="padding:10px;background:#fee2e2;border-radius:4px;border-left:4px solid #ef4444;color:#991b1b;"><b>🛑 Kernel changed</b><br>Preflight was run under:<br><code>{WHISPERJAV_PREFLIGHT.get("python")}</code><br>Current kernel is:<br><code>{sys.executable}</code><br>Re-run <b>Step 1.6</b>.</div>'))
    raise SystemExit()

def tail_lines(path: Optional[Path], n: int = 30, max_chars: int = 4000) -> str:
    """Return the last N lines of a text file (best-effort) for notebook status panels."""
    if not path or not path.exists():
        return ""
    try:
        text = path.read_text(encoding="utf-8", errors="replace")
    except Exception:
        try:
            text = path.read_text(encoding="utf-8")
        except Exception:
            return ""
    lines = text.splitlines()
    tail = "\n".join(lines[-n:])
    if max_chars and len(tail) > max_chars:
        tail = tail[-max_chars:]
    return tail

def render_pass_panel(pass_states: Dict[int, Dict[str, Any]], tail_n: int = 4) -> None:
    """Render a compact, overwrite-in-place status panel for Kaggle/Colab notebooks."""
    clear_output(wait=True)
    section("EXECUTION (LIVE STATUS)")

    for pass_num in sorted(pass_states.keys()):
        st = pass_states[pass_num]
        status_txt = st.get("status", "?")
        elapsed = st.get("elapsed", 0.0)
        rc = st.get("returncode")
        log_path = st.get("log_path")
        extra = ""
        if rc is not None:
            extra = f" | rc={rc}"
        print(f"Pass {pass_num}: {status_txt} | {elapsed:.0f}s{extra}")
        if log_path:
            tail = tail_lines(Path(log_path), n=tail_n)
            if tail:
                print("--- log tail ---")
                print(tail)
        print()

# Check config
if 'WHISPERJAV_CONFIG' not in dir():
    display(HTML('<div style="padding:8px;background:#fef2f2;border-radius:4px;color:#991b1b;font-size:10px"><b>Error:</b> Run Step 1 first</div>'))
    raise SystemExit()
cfg = WHISPERJAV_CONFIG

# Check for expert config (always present now)
expert = WHISPERJAV_EXPERT_CONFIG if 'WHISPERJAV_EXPERT_CONFIG' in dir() else None

# ═══════════════════════════════════════════
# PLATFORM DETECTION & FILE SYSTEM SETUP
# ═══════════════════════════════════════════
section("DETECTING PLATFORM & PATHS")

def detect_platform():
    if 'google.colab' in sys.modules:
        return 'colab'
    elif os.path.exists('/kaggle'):
        return 'kaggle'
    else:
        return 'local'

PLATFORM = detect_platform()
print(f"Platform: {PLATFORM.upper()}")

if PLATFORM == 'colab':
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=False)
        base_dir = Path(f"/content/drive/MyDrive")
        input_dir = base_dir / cfg['folder_name']
        output_dir = input_dir # Colab writes back to same folder
        input_dir.mkdir(parents=True, exist_ok=True)
        status(f"Drive Connected: {input_dir}")
    except Exception as e:
        status(f"Failed to connect Drive: {e}", False)
        raise SystemExit()

elif PLATFORM == 'kaggle':
    # Kaggle: Read-only Input, specific Output
    # Search for input dataset matching folder_name or standard locations
    potential_inputs = [
        Path(f"/kaggle/input/{cfg['folder_name']}"), # Named dataset
        Path(f"/kaggle/input/whisperjav-media"),    # Common dataset name
        Path(f"/kaggle/input"),                     # Root input
        Path(f"/kaggle/working/input")              # Manual upload location
    ]
    
    input_dir = None
    for p in potential_inputs:
        if p.exists() and any(f for f in p.rglob('*') if f.suffix.lower() in {'.mp4', '.mkv', '.avi', '.mp3', '.wav', '.flac'}):
            input_dir = p
            break
            
    if not input_dir:
        input_dir = Path(f"/kaggle/input")
        print("  ⚠ Could not auto-detect specific input folder. Scanning /kaggle/input root.")

    output_dir = Path(f"/kaggle/working/{cfg['folder_name']}")
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # KAGGLE FIX 1: Fix Temp Storage Overflow
    # Redirect all temp files to writable working dir to avoid /tmp (ramdisk) filling up
    kaggle_temp = Path(f"/kaggle/working/temp")
    kaggle_temp.mkdir(parents=True, exist_ok=True)
    os.environ['TMPDIR'] = str(kaggle_temp)
    os.environ['TEMP'] = str(kaggle_temp)
    os.environ['TMP'] = str(kaggle_temp)
    print(f"  🔧 Redirected temp storage to {kaggle_temp}")

    # KAGGLE CHECK 2: Internet Access
    try:
        import socket
        socket.create_connection(("www.google.com", 80), timeout=2)
    except OSError:
        display(HTML('<div style="padding:10px;background:#fee2e2;border-radius:4px;border-left:4px solid #ef4444;color:#991b1b;"><b>🛑 Internet Disabled!</b><br>You must enable "Internet" in the Notebook Settings (right sidebar) > Internet > On.<br>Otherwise installation will fail.</div>'))
        time.sleep(5)

    status(f"Input: {input_dir}")
    status(f"Output: {output_dir}")

else:
    # Local fallback
    input_dir = Path(cfg['folder_name'])
    output_dir = Path(f"{cfg['folder_name']}_output")
    output_dir.mkdir(parents=True, exist_ok=True)
    status(f"Local Mode: {input_dir} → {output_dir}")

# ═══════════════════════════════════════════
# CHECK GPUs AND DETERMINE MODE
# ═══════════════════════════════════════════
section("GPU RESOURCE CHECK")

# Fail-fast: this notebook is intended for GPU runtimes (Kaggle/Colab).
gpu_check = subprocess.run(
    "nvidia-smi --query-gpu=name,memory.total --format=csv,noheader",
    shell=True,
    capture_output=True,
    text=True,
 )

if gpu_check.returncode != 0 or not gpu_check.stdout.strip():
    status("No GPU detected. Switch runtime to GPU (T4 x2 or L4/A100).", False)
    if PLATFORM == 'kaggle':
        print("  Kaggle: right sidebar → Settings → Accelerator → GPU (T4 x2) → Save.")
    elif PLATFORM == 'colab':
        print("  Colab: Runtime → Change runtime type → Hardware accelerator: GPU.")
    else:
        print("  Local: ensure NVIDIA drivers are installed and `nvidia-smi` works.")
    raise SystemExit()

gpu_lines = [line.strip() for line in gpu_check.stdout.strip().split('\n') if line.strip()]
num_gpus = len(gpu_lines)

for i, gpu_info in enumerate(gpu_lines):
    status(f"GPU {i}: {gpu_info}")

# Adaptive mode selection
if num_gpus >= 2:
    PARALLEL_MODE = True
    gpu_assignment = {1: "0", 2: "1"}
    print(f"\n  ⚡ Parallel Mode: Pass 1 (GPU 0) | Pass 2 (GPU 1)")
else:
    PARALLEL_MODE = False
    gpu_assignment = {1: "0", 2: "0"}
    print(f"\n  📝 Sequential Mode: Pass 1 → Pass 2 (Single GPU)")

# ═══════════════════════════════════════════
# INSTALL WHISPERJAV
# ═══════════════════════════════════════════
section("INSTALLING (3-5 min)")
install_start = time.time()

# NOTE: TEN VAD depends on system libc++ runtime.
# These installs must succeed (and we verify them) or TEN will fail at runtime.
def _apt_prefix():
    try:
        if hasattr(os, 'geteuid') and os.geteuid() == 0:
            return ''
    except Exception:
        pass
    return 'sudo ' if shutil.which('sudo') else ''

# Platform prereqs (Colab/Kaggle): include libc++ runtime for TEN
pre_cmds = (
    f"{_apt_prefix()}apt-get update -qq && "
    f"{_apt_prefix()}apt-get install -y -qq ffmpeg portaudio19-dev libc++1 libc++abi1 > /dev/null 2>&1"
 )

# ARCHITECTURE NOTE:
# WhisperJAV requires NumPy>=2.0. Standard Kaggle/Colab environments use NumPy 1.x.
# To satisfy the requirement without breaking the ecosystem (matplotlib/torch ABI mismatches),
# we must perform a synchronized upgrade of the core stack (numpy, matplotlib, torch).
# We rely on 'configure_cuda_runtime_paths()' (below) to fix any CUDA version mismatches 
# caused by replacing the system PyTorch.

req_packages = [
# Core Audio/Video
"ffmpeg-python", "auditok", "pysrt", "srt", "aiofiles", "pyloudnorm", "pydub",
# AI/ML Stack (Synchronized Upgrade)
"numpy", "matplotlib", "scipy", "torch", "torchaudio", "torchvision",
"faster-whisper", "transformers", "optimum", "accelerate", "huggingface-hub", 
"pydantic", "ten-vad", "silero-vad", "modelscope", "addict", "tiktoken"
 ]
req_str = " ".join(req_packages)

steps = [
    (pre_cmds, "System tools"),
    # Install dependencies allowing upgrades to ensure consistency (Numpy 2.0 compatible stack)
    (f"pip install -q {req_str}", "Python packages (Core Stack)"),
    ("pip install -q --no-deps git+https://github.com/openai/whisper.git@main", "Whisper"),
    ("pip install -q --no-deps git+https://github.com/meizhong986/stable-ts-fix-setup.git@main", "Stable-TS"),
    ("pip install -q --no-deps git+https://github.com/meizhong986/WhisperJAV.git@main", "WhisperJAV")
]

for cmd, name in steps:
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        status(f"{name} failed", False)
        # On Kaggle, some apt-get might fail but environment is often usable
        if PLATFORM != 'kaggle': 
            raise SystemExit()
    else:
        status(name)

# Conditional installation of speech enhancer dependencies
if expert:
    extra_packages = set()
    for enhancer in [expert.get('pass1_speech_enhancer'), expert.get('pass2_speech_enhancer')]:
        if enhancer == 'clearvoice':
            extra_packages.add('clearvoice')
        elif enhancer == 'zipenhancer':
            # zipenhancer uses modelscope which is already installed above
            pass
        elif enhancer == 'bs-roformer':
            extra_packages.add('bs-roformer-infer')
    
    if extra_packages:
        pkg_list = ' '.join(extra_packages)
        result = subprocess.run(f"pip install -q {pkg_list}", shell=True, capture_output=True, text=True)
        if result.returncode != 0:
            status(f"Speech enhancer packages failed (continuing anyway)", False)
        else:
            status(f"Speech enhancer packages ({', '.join(extra_packages)})")

status(f"Installation complete ({time.time()-install_start:.0f}s)")

# ═══════════════════════════════════════════
# RUNTIME CONFIGURATION (Fixing CUDA conflicts)
# ═══════════════════════════════════════════
# ... existing code continues unchanged ...

In [ ]:
#@title Step 3: AI Translation (if selected) { display-mode: "form" }
#@markdown Translate each subtitle file using AI (only runs if "English (AI translate)" selected)

import os, sys, subprocess, shlex, time
from pathlib import Path
from IPython.display import display, HTML

def status(msg, ok=True):
    icon = "✓" if ok else "✗"
    print(f"{icon} {msg}")

def section(title):
    print(f"\n{'─'*40}\n{title}\n{'─'*40}")

# Check prerequisites
if 'WHISPERJAV_CONFIG' not in dir():
    display(HTML('<div style="padding:8px;background:#fef2f2;border-radius:4px;color:#991b1b;font-size:10px"><b>Error:</b> Run Step 1 first</div>'))
    raise SystemExit()

if 'WHISPERJAV_NEW_SRTS' not in dir():
    display(HTML('<div style="padding:8px;background:#fef2f2;border-radius:4px;color:#991b1b;font-size:10px"><b>Error:</b> Run Step 2 first</div>'))
    raise SystemExit()

cfg = WHISPERJAV_CONFIG
new_srts = WHISPERJAV_NEW_SRTS
output_dir = WHISPERJAV_OUTPUT_DIR if 'WHISPERJAV_OUTPUT_DIR' in dir() else Path(os.getcwd())

# Re-detect platform just in case
if 'google.colab' in sys.modules: PLATFORM = 'colab'
elif os.path.exists('/kaggle'): PLATFORM = 'kaggle'
else: PLATFORM = 'local'

# Check if AI translation is needed
if cfg['subtitle_language'] != 'llm':
    display(HTML('<div style="padding:8px 10px;background:#f0f9ff;border-radius:4px;border-left:2px solid #3b82f6;font-size:10px"><b>ℹ Skipped:</b> AI translation not selected</div>'))
    raise SystemExit()

# ═══════════════════════════════════════════
# API KEY RESOLUTION (SECRETS MANAGER)
# ═══════════════════════════════════════════
def retrieve_key(service_name, config_key):
    """Try to get key from config, Kaggle secrets, or env."""
    if config_key and config_key.strip():
        return config_key.strip()
    
    # Try Kaggle Secrets
    if PLATFORM == 'kaggle':
        try:
            from kaggle_secrets import UserSecretsClient
            secrets = UserSecretsClient()
            candidates = [
                f"{service_name.upper()}_API_KEY",
                "API_KEY",
                "LLM_API_KEY",
                service_name.lower()
            ]
            for c in candidates:
                try:
                    k = secrets.get_secret(c)
                    if k: return k
                except: pass
        except: pass
        
    return ""

final_api_key = retrieve_key(cfg['translation_service'], cfg.get('api_key'))

if not final_api_key:
    display(HTML('<div style="padding:8px;background:#fef2f2;border-radius:4px;color:#991b1b;font-size:10px"><b>Error:</b> No API key provided (checked Config and Kaggle Secrets)</div>'))
    raise SystemExit()

if not new_srts:
    display(HTML('<div style="padding:8px;background:#fef2f2;border-radius:4px;color:#991b1b;font-size:10px"><b>Error:</b> No subtitle files to translate</div>'))
    raise SystemExit()

# Set up API key
env_map = {
    "deepseek": "DEEPSEEK_API_KEY",
    "openrouter": "OPENROUTER_API_KEY",
    "gemini": "GEMINI_API_KEY",
    "claude": "ANTHROPIC_API_KEY",
    "gpt": "OPENAI_API_KEY"
}
os.environ[env_map.get(cfg['translation_service'], "API_KEY")] = final_api_key

# ═══════════════════════════════════════════
# TRANSLATION LOOP
# ═══════════════════════════════════════════
section("AI TRANSLATION")
print(f"Provider: {cfg['translation_service']}")
print(f"Style: {cfg['_translation_style']}")
print(f"Files to translate: {len(new_srts)}\n")

translated_files = []
failed_files = []

for i, srt_file in enumerate(new_srts, 1):
    print(f"[{i}/{len(new_srts)}] Translating: {srt_file.name}")

    translate_cmd = [
        'whisperjav-translate',
        '-i', str(srt_file),
        '--provider', cfg['translation_service'],
        '-t', 'english',
        '--tone', cfg['translation_style'],
        '--stream'
    ]

    full_cmd = shlex.join(translate_cmd)

    try:
        process = subprocess.Popen(
            full_cmd,
            shell=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            bufsize=1,
            universal_newlines=True
        )

        for line in process.stderr:
            print(f"    {line}", end='')

        stdout_output, _ = process.communicate()

        if process.returncode == 0:
            output_path = stdout_output.strip()
            if output_path:
                translated_files.append(Path(output_path))
            status(f"Completed: {srt_file.name}")
        else:
            status(f"Failed: {srt_file.name}", False)
            failed_files.append(srt_file)

    except Exception as e:
        status(f"Error translating {srt_file.name}: {e}", False)
        failed_files.append(srt_file)

    print()

# ═══════════════════════════════════════════
# COMPLETE
# ═══════════════════════════════════════════
section("COMPLETE")

# Create ZIP of translations
if translated_files:
    trans_zip_path = output_dir / f"{cfg['folder_name']}_translations_{cfg['translation_service']}.zip"
    try:
        import zipfile
        with zipfile.ZipFile(trans_zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for srt in translated_files:
                zipf.write(srt, srt.name)
        status(f"Bundled translations into: {trans_zip_path.name}")
        
        # Download triggers
        if PLATFORM == 'kaggle':
            try:
                from IPython.display import FileLink
                rel_path = trans_zip_path.relative_to(Path.cwd())
                display(FileLink(str(rel_path), result_html_prefix="<b>⬇ Click to download Translations ZIP: </b>", result_html_suffix=" (Kaggle)"))
            except:
                display(HTML(f'<b>⬇ <a href="{trans_zip_path.name}" target="_blank">Click here to download Translations ZIP</a></b> (Kaggle)'))

        elif PLATFORM == 'colab':
            try:
                from google.colab import files
                files.download(str(trans_zip_path))
            except: pass
            
    except Exception as e:
        print(f"Failed to create zip: {e}")

if failed_files:
    display(HTML(f'<div style="padding:8px 10px;background:#fef9c3;border-radius:4px;border-left:2px solid #ca8a04;font-size:10px"><b>⚠ Partially done!</b> {len(translated_files)}/{len(new_srts)} translated. {len(failed_files)} failed.</div>'))
else:
    if PLATFORM == 'kaggle':
        loc_msg = f"<br>📂 <b>files in 'Output' tab → <code>{cfg['folder_name']}/</code></b>. <br>⚠ <b>Download now</b> or they will be lost when session ends!"
    else:
        loc_msg = f"in {output_dir}"

    display(HTML(f'<div style="padding:8px 10px;background:#f0fdf4;border-radius:4px;border-left:2px solid #16a34a;font-size:10px"><b>✓ All done!</b> {len(new_srts)} Japanese + {len(translated_files)} English subtitle(s) {loc_msg}</div>'))

# Auto-disconnect
if cfg['auto_disconnect']:
    print("\nAuto-disconnecting in 10s...")
    time.sleep(10)
    if PLATFORM == 'colab':
        try:
            from google.colab import runtime
            runtime.unassign()
        except: pass
else:
    print("\nRemember to disconnect manually.")
